Connect to Azure Open AI API 

In [ ]:
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity

load_dotenv()

openai.api_type = "azure"
openai.api_version = "2023-07-01-preview" 
openai.api_base = os.getenv('OPENAI_API_BASE')  
openai.api_key = os.getenv('OPENAI_API_KEY') 

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", engine="text-embedding-ada-002", chunk_size=1)

Generating the first embeddings

In [ ]:
e = embeddings.embed_query("Azure Bootcamp", )
print(e)

Cosine similarity measures the similarity between two vectors of an inner product space.

In [ ]:
def calculateSimilarity(word1, word2):
    e1 = embeddings.embed_query(word1)
    e2 = embeddings.embed_query(word2)
    print (f'{word1} vs {word2}  \nSimilarity: {cosine_similarity(e1, e2)}')

In [ ]:
calculateSimilarity("King","Queen")
calculateSimilarity("King","Beggar")
calculateSimilarity("King","König")

In [ ]:
calculateSimilarity("Madrid","Barcelona")

# vs. Thun, Zurich, Singapore

In [ ]:
calculateSimilarity("SC Bern","Schweizermeister")
calculateSimilarity("HC Fribourg-Gottéron","Schweizermeister")
calculateSimilarity("FC Basel","Schweizermeister")

Import a list of sessions of the Azure Bootcamp Switzerland 2023 and find similar sessions

In [ ]:
import pandas as pd

sessions = pd.read_csv('./sessions.csv')
sessions.head()

Add a new column and calculate the embeddings.

In [ ]:
sessions['embedding'] = sessions['Content'].apply(lambda x: embeddings.embed_query(x))
sessions.head()

Find similar sessions

In [ ]:
preffered_session = "powerbi" # "avnm" "quantum" "powerbi"
e = sessions[sessions['Key'] == preffered_session]['embedding'].values[0]

similar_sessions = sessions['embedding'].apply(lambda x: cosine_similarity(x, e))

recommandations=pd.concat([sessions['Title'],sessions['Speaker'],similar_sessions], axis=1).sort_values(by='embedding', ascending=False)
recommandations.head()